In [9]:
import data_process as dp
import data_graph as dg

# establish universal regions, plus a scrap bin
region_list = ["w", "m", "s", "n"]

# request dataset from online and store it in a variable
# data_fetch.write_to_csv(
# "https://www.ncei.noaa.gov/archive/archive-management-system/OAS/bin/prd/jquery/download/209268.17.17.tar.gz",
# "209268.17.17.tar.gz",
# )

disaster_data = dp.read_csv_to_var(
    "./0209268/17.17/data/0-data/events-US-1980-2023.csv"
)

# modify dates to be less specific years
dp.parse_all_years(disaster_data)

# universal variables
all_years = dp.retrieve_unique_years(disaster_data)
all_disaster_types = dp.retrieve_unique_disaster_types(disaster_data)
year_buckets = [] # TO FILL. buckets to group time data by (one/four/five/ten-year groups)

# convert the raw data (sorted by region) into graphable blocks of data
region_dict = dp.fill_all_regions(disaster_data, region_list)
cost_of_regions, deaths_of_regions = dp.organize_regions(region_dict, all_years, all_disaster_types)


*********Sorting w*********
*********Sorting m*********
*********Sorting s*********
*********Sorting n*********


now we graph!

{'Flooding': [0, 0, 4693.8, 1220.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5100.0, 2006.5, 5822.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1976.1, 0, 0, 0, 1890.0, 0, 0, 0, 1281.2, 4635.0, 0], 'Tropical Cyclone': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6727.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Drought': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5378.0, 0, 0, 0, 0, 0, 0, 5176.8, 5850.6, 0, 0, 0, 0, 0, 9864.5, 0, 0], 'Freeze': [0, 0, 0, 0, 0, 0, 0, 0, 0, 8194.0, 0, 0, 0, 0, 0, 0, 0, 4750.0, 0, 0, 0, 0, 0, 0, 0, 0, 2128.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Severe Storm': [0, 0, 0, 0, 0, 0, 1488.8, 0, 0, 1927.1, 0, 0, 0, 0, 0, 0, 1908.5, 0, 0, 0, 0, 0, 0, 1125.9, 0, 0, 0, 0, 1430.0, 5320.0, 0, 0, 0, 0, 0, 0, 0, 4915.4, 1209.6, 0, 0, 0, 0], 'Winter Storm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Wildf